# Assignment 4

In [2]:
import numpy as np
import pandas as pd
import spacy
from spacy import displacy
from spacy.tokens import Doc
import json
from datasets import load_dataset

In [3]:
df = pd.read_csv("ner.csv")
df.head()

,Sentence #,Sentence,POS,Tag
0,Sentence: 1,Thousands of demonstrators have marched throug...,"['NNS', 'IN', 'NNS', 'VBP', 'VBN', 'IN', 'NNP'...","['O', 'O', 'O', 'O', 'O', 'O', 'B-geo', 'O', '..."
1,Sentence: 2,Families of soldiers killed in the conflict jo...,"['NNS', 'IN', 'NNS', 'VBN', 'IN', 'DT', 'NN', ...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
2,Sentence: 3,They marched from the Houses of Parliament to ...,"['PRP', 'VBD', 'IN', 'DT', 'NNS', 'IN', 'NN', ...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
3,Sentence: 4,"Police put the number of marchers at 10,000 wh...","['NNS', 'VBD', 'DT', 'NN', 'IN', 'NNS', 'IN', ...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
4,Sentence: 5,The protest comes on the eve of the annual con...,"['DT', 'NN', 'VBZ', 'IN', 'DT', 'NN', 'IN', 'D...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."


In [4]:
df = df.drop(["Tag"], axis=1)

In [5]:
df.head(5)

,Sentence #,Sentence,POS
0,Sentence: 1,Thousands of demonstrators have marched throug...,"['NNS', 'IN', 'NNS', 'VBP', 'VBN', 'IN', 'NNP'..."
1,Sentence: 2,Families of soldiers killed in the conflict jo...,"['NNS', 'IN', 'NNS', 'VBN', 'IN', 'DT', 'NN', ..."
2,Sentence: 3,They marched from the Houses of Parliament to ...,"['PRP', 'VBD', 'IN', 'DT', 'NNS', 'IN', 'NN', ..."
3,Sentence: 4,"Police put the number of marchers at 10,000 wh...","['NNS', 'VBD', 'DT', 'NN', 'IN', 'NNS', 'IN', ..."
4,Sentence: 5,The protest comes on the eve of the annual con...,"['DT', 'NN', 'VBZ', 'IN', 'DT', 'NN', 'IN', 'D..."


In [6]:
df['Sentence #'].value_counts()

Sentence #
Sentence: 1        1
Sentence: 31978    1
Sentence: 31969    1
Sentence: 31970    1
Sentence: 31971    1
                  ..
Sentence: 15989    1
Sentence: 15990    1
Sentence: 15991    1
Sentence: 15992    1
Sentence: 47959    1
Name: count, Length: 47959, dtype: int64

In [7]:
sample_df = df.head(210).copy()

## SpaCy en_core_web_sm

In [9]:
# Function to get Doccano-style entities
nlp = spacy.load("en_core_web_sm")

def get_spacy_entities(text):
    doc = nlp(text)
    return [[ent.start_char, ent.end_char, ent.label_] for ent in doc.ents]

# Create Doccano-ready columns
sample_df["text"] = sample_df["Sentence"]
sample_df["label"] = sample_df["Sentence"].apply(get_spacy_entities)

# Only keep relevant columns
spacy_df = sample_df[["text", "label"]]

In [10]:
def spacy_export(df, output_path="spacy_ner.jsonl"):
    with open(output_path, "w", encoding="utf-8") as f:
        for _, row in df.iterrows():
            text = row["text"]
            labels = row["label"] if isinstance(row["label"], list) else []
            json.dump({"text": text, "label": labels}, f)
            f.write("\n")

# Usage
spacy_export(spacy_df)


## First Model with Annotated Data

In [12]:

def jsonl_to_conll_variable(input_file):
    """
    Convert a JSONL file to a CONLL formatted string variable for NER training.

    Each JSON object in the JSONL file should have:
      - "text": The raw text.
      - "label": A list of entity annotations in the format [start, end, label].

    Returns:
      A string containing the CONLL formatted data.
    """
    # Load spaCy model for tokenization and sentence segmentation.
    nlp = spacy.load("en_core_web_sm")

    conll_lines = []

    with open(input_file, 'r', encoding='utf-8') as infile:
        for line in infile:
            line = line.strip()
            if not line:
                continue
            data = json.loads(line)
            text = data.get("text", "")
            # Expecting "label" to be a list of lists: [start, end, entity_label]
            labels = data.get("label", [])

            # Process the text using spaCy.
            doc = nlp(text)

            # Process each sentence in the document.
            for sent in doc.sents:
                for token in sent:
                    token_tag = "O"
                    # Check if the token's span is within any annotated entity span.
                    for start, end, ent_label in labels:
                        if token.idx >= start and (token.idx + len(token)) <= end:
                            # Mark token as beginning (B-) or inside (I-) of an entity.
                            if token.idx == start:
                                token_tag = f"B-{ent_label}"
                            else:
                                token_tag = f"I-{ent_label}"
                            break
                    conll_lines.append(f"{token.text} {token_tag}")
                # Append a blank line to separate sentences.
                conll_lines.append("")

    # Join all lines into a single string.
    conll_text = "\n".join(conll_lines)
    return conll_text

In [13]:
# from JSONL again to CONLL , ready for training
input_file = "train_data.jsonl"  # Replace with the path to your JSONL file.
conll_output = jsonl_to_conll_variable(input_file)
print(conll_output)  # You can now use conll_output for NER training.

Thousands B-CARDINAL
of O
demonstrators O
have O
marched O
through O
London B-GPE
to O
protest O
the O
war O
in O
Iraq B-GPE
and O
demand O
the O
withdrawal O
of O
British B-NORP
troops O
from O
that O
country O
. O

Families O
of O
soldiers O
killed O
in O
the O
conflict O
joined O
the O
protesters O
who O
carried O
banners O
with O
such O
slogans O
as O
" O
Bush B-PERSON
Number O
One O
Terrorist O
" O
and O
" O
Stop O
the O
Bombings O
. O
" O

They O
marched O
from O
the B-ORG
Houses I-ORG
of I-ORG
Parliament I-ORG
to O
a O
rally O
in O
Hyde B-GPE
Park I-GPE
. O

Police O
put O
the O
number O
of O
marchers O
at O
10,000 B-CARDINAL
while O
organizers O
claimed O
it O
was O
1,00,000 B-CARDINAL
. O

The O
protest O
comes O
on O
the O
eve O
of O
the O
annual O
conference O
of O
Britain B-GPE
's O
ruling O
Labor B-ORG
Party I-ORG
in O
the O
southern O
English B-NORP
seaside O
resort O
of O
Brighton B-GPE
. O

The O
party O
is O
divided O
over O
Britain B-GPE
's O
participation O
in O
the 

In [14]:
with open('conll_output.txt', 'w', encoding='utf-8') as f:
    f.write(conll_output)

In [15]:
def read_conll(filename):
    """Read CoNLL-formatted data from a file."""
    sentences = []
    with open(filename, encoding="utf-8") as f:
        sentence = []
        for line in f:
            line = line.strip()
            if not line:
                if sentence:
                    sentences.append(sentence)
                    sentence = []
            else:
                parts = line.split()
                # Use the first column as the word and the last column as the label.
                word = parts[0]
                label = parts[-1]
                sentence.append((word, label))
        if sentence:
            sentences.append(sentence)
    return sentences

def word2features(sent, i):
    """Extract simple features for the word at position i in the sentence."""
    word = sent[i][0]
    features = {
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
    }
    # Add previous word's feature if available
    if i > 0:
        features['prev.word.lower()'] = sent[i-1][0].lower()
    else:
        features['BOS'] = True  # Beginning of Sentence
    # Add next word's feature if available
    if i < len(sent) - 1:
        features['next.word.lower()'] = sent[i+1][0].lower()
    else:
        features['EOS'] = True  # End of Sentence
    return features

def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for _, label in sent]

In [16]:
!pip install sklearn_crfsuite

In [17]:
from sklearn.model_selection import train_test_split
from sklearn_crfsuite import CRF

filename = "conll_output.txt"
sentences = read_conll(filename)
print(f"Read {len(sentences)} sentences from {filename}.")

train_sents, test_sents = train_test_split(sentences, test_size=0.2, random_state=42)
print(f"Number of training sentences: {len(train_sents)}")
print(f"Number of test sentences: {len(test_sents)}")

# Prepare the features and labels for both sets
X_train = [sent2features(s) for s in train_sents]
y_train = [sent2labels(s) for s in train_sents]

X_test = [sent2features(s) for s in test_sents]
y_test = [sent2labels(s) for s in test_sents]

# Initialize and train the CRF model
crf = CRF(algorithm='lbfgs', max_iterations=50)
crf.fit(X_train, y_train)

# Predict on the training set
y_pred = crf.predict(X_train)

Read 301 sentences from conll_output.txt.
Number of training sentences: 240
Number of test sentences: 61


### Training Accuracy

In [18]:
from sklearn_crfsuite import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

# Compute and print training accuracy
accuracy = metrics.flat_accuracy_score(y_train, y_pred)
print("Training Accuracy: {:.4f}".format(accuracy))

# Optionally, print a detailed classification report
report = metrics.flat_classification_report(y_train, y_pred)
print(report)

Training Accuracy: 0.9744
              precision    recall  f1-score   support

  B-CARDINAL       0.94      0.67      0.78        97
       B-GPE       0.96      0.96      0.96       198
      B-NORP       0.97      0.90      0.93        98
       B-ORG       0.88      0.70      0.78       105
    B-PERSON       0.97      0.85      0.91        73
  I-CARDINAL       1.00      0.50      0.67        16
       I-GPE       1.00      0.78      0.88        37
      I-NORP       1.00      0.33      0.50         3
       I-ORG       0.91      0.93      0.92       115
    I-PERSON       0.94      1.00      0.97        60
           O       0.98      1.00      0.99      4510

    accuracy                           0.97      5312
   macro avg       0.96      0.78      0.84      5312
weighted avg       0.97      0.97      0.97      5312



### Testing Accuracy

In [19]:
# Predict on the training set
y_pred = crf.predict(X_test)
# Compute and print training accuracy
accuracy = metrics.flat_accuracy_score(y_test, y_pred)
print("Test Accuracy: {:.4f}".format(accuracy))

# Optionally, print a detailed classification report
report = metrics.flat_classification_report(y_test, y_pred)
print(report)

Test Accuracy: 0.9250
              precision    recall  f1-score   support

  B-CARDINAL       0.80      0.55      0.65        22
       B-GPE       0.89      0.71      0.79        59
      B-NORP       0.91      0.81      0.86        26
       B-ORG       0.72      0.41      0.52        32
    B-PERSON       0.68      0.59      0.63        22
  I-CARDINAL       0.00      0.00      0.00         2
       I-GPE       0.80      0.25      0.38        16
      I-NORP       0.00      0.00      0.00         2
       I-ORG       0.68      0.64      0.66        44
    I-PERSON       0.68      0.88      0.77        17
           O       0.95      0.99      0.97      1171

    accuracy                           0.92      1413
   macro avg       0.65      0.53      0.57      1413
weighted avg       0.92      0.92      0.92      1413



C:\Users\matth\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\matth\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\matth\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


### Classification Report

In [20]:
def aggregate_label(label):
    """
    Remove the B- or I- prefix from a label.
    Returns the base entity (e.g., "B-corporation" or "I-corporation" becomes "corporation").
    The "O" label remains unchanged.
    """
    if label == "O":
        return label
    if "-" in label:
        return label.split("-", 1)[1]
    return label

# Build aggregated lists by iterating pairwise over each sentence.
y_true_agg = []
y_pred_agg = []
for true_sent, pred_sent in zip(y_test, y_pred):
    for true_label, pred_label in zip(true_sent, pred_sent):
        # Only include tokens where the true label is not "O"
        if true_label != "O":
            y_true_agg.append(aggregate_label(true_label))
            y_pred_agg.append(aggregate_label(pred_label))

# Determine the list of non-"O" labels from the aggregated true labels.
non_o_labels = sorted(set(y_true_agg))

# Now generate the classification report using only the non-"O" labels.
report = classification_report(y_true_agg, y_pred_agg, labels=non_o_labels, digits=4)
print("Aggregated Classification Report (without O tokens):")
print(report)


Aggregated Classification Report (without O tokens):
              precision    recall  f1-score   support

    CARDINAL     1.0000    0.5417    0.7027        24
         GPE     0.9412    0.6400    0.7619        75
        NORP     0.9130    0.7500    0.8235        28
         ORG     0.7500    0.5526    0.6364        76
      PERSON     0.8108    0.7692    0.7895        39

   micro avg     0.8556    0.6364    0.7299       242
   macro avg     0.8830    0.6507    0.7428       242
weighted avg     0.8627    0.6364    0.7282       242



In [21]:
# Flatten and aggregate the labels.
y_true_flat = [aggregate_label(label) for sent in y_test for label in sent]
y_pred_flat = [aggregate_label(label) for sent in y_pred for label in sent]

# Determine the sorted set of aggregated labels.
labels = sorted(list(set(y_true_flat + y_pred_flat)))

# Compute the confusion matrix.
cm = confusion_matrix(y_true_flat, y_pred_flat, labels=labels)
df_cm = pd.DataFrame(cm, index=labels, columns=labels)

print("\nAggregated Confusion Matrix:")
df_cm


Aggregated Confusion Matrix:


,CARDINAL,GPE,NORP,O,ORG,PERSON
CARDINAL,13,0,0,10,0,1
GPE,0,48,1,15,9,2
NORP,0,0,21,3,3,1
O,4,1,0,1159,3,4
ORG,0,2,1,28,42,3
PERSON,0,1,0,6,2,30


### Export to Annotate Again

In [22]:
doccano_data = []

# Function to convert to Doccano format
def convert_to_doccano(sents, labels):
    data = []
    for sent, true_labels in zip(sents, labels):
        tokens = [token for token, label in sent]
        sentence_text = " ".join(tokens)

        offset = 0
        entities = []

        for token, label in zip(tokens, true_labels):
            start_pos = offset
            end_pos = start_pos + len(token)

            if label != "O":  # Only keep entities, skip non-entity tokens
                entity_label = label.split("-")[-1]  # Handle BIO format
                entities.append([start_pos, end_pos, entity_label])

            offset = end_pos + 1  # +1 for space

        data.append({
            "text": sentence_text,
            "label": entities
        })
    return data

# Convert training and test data
doccano_data += convert_to_doccano(train_sents, y_train)
doccano_data += convert_to_doccano(test_sents, y_test)


In [23]:
def write_jsonl(data, output_file):
    """
    Write a list of dictionaries to a JSONL file.

    Each dictionary is written as a separate JSON object on a new line.
    """
    with open(output_file, "w", encoding="utf-8") as outfile:
        for entry in data:
            outfile.write(json.dumps(entry) + "\n")
    print(f"Annotated data saved to {output_file}")
write_jsonl(doccano_data, 'annotations_2.jsonl')

Annotated data saved to annotations_2.jsonl


## Second Model with Annotated Data

In [25]:
# Retrieve the entity label names from the dataset
labels = dataset["train"].features["ner_tags"].feature.names

# Function to convert numerical NER tags into human-readable labels
def decode_ner_tags(example):
    example["ner_labels"] = [labels[tag] for tag in example["ner_tags"]]
    return example

# Apply the function to all data splits
dataset = dataset.map(decode_ner_tags)

NameError: name 'dataset' is not defined

In [ ]:
print("Entity Types in the Dataset:", labels)


In [ ]:
train_dataset = dataset["train"]
df = train_dataset.to_pandas()

In [ ]:
# Count unique document IDs
num_documents = df["doc_id"].nunique()
print(f"Number of unique documents: {num_documents}")

In [ ]:
import spacy
from spacy.tokens import Doc
from spacy import displacy

# Load a blank spaCy model
nlp = spacy.blank("en")

# Function to visualize Named Entities
def visualize_ner(example):
    tokens = example["tokens"]
    labels = example["ner_labels"]
    start_chars = example["start_char"]
    end_chars = example["end_char"]

    # Convert entity positions into spaCy's expected format
    ents = []
    for start, end, label in zip(start_chars, end_chars, labels):
        if label != "O":  # Ignore "O" (Outside) labels
            ents.append(nlp.make_doc(example["sentence"]).char_span(start, end, label=label))

    # Create a spaCy Doc object
    doc = nlp.make_doc(example["sentence"])

    # Assign named entities (filter out None values)
    doc.ents = [ent for ent in ents if ent is not None]

    # Render visualization in Jupyter Notebook
    displacy.render(doc, style="ent", jupyter=True)

# Display the first example with highlighted named entities
visualize_ner(dataset["train"][1])

In [ ]:
# Function to apply the pretrained model and display structured results
def evaluate_ner_predictions(example):
    text = example["sentence"]
    doc = nlp_core(text)  # Use pretrained model
    predictions = [(ent.text, ent.label_) for ent in doc.ents]  # Extract model-predicted entities

    # Compare with dataset labels
    expected_entities = list(zip(example["tokens"], example["ner_labels"]))

    # Convert to DataFrame for better readability
    df_results = pd.DataFrame(predictions, columns=["Text Span", "Model Prediction"])
    df_expected = pd.DataFrame(expected_entities, columns=["Text Span", "Expected Label"])

    # Merge for side-by-side comparison
    results_df = df_results.merge(df_expected, on="Text Span", how="outer").fillna("N/A")

    print(results_df)

# Test on a sample
evaluate_ner_predictions(dataset["train"][1])

In [ ]:
# Function to process text and extract entities
def get_entities_from_text(text):
    # Process the text with spaCy model
    doc = nlp(text)
    # Return a list of tuples with entity text and label
    return [(ent.text, ent.label_) for ent in doc.ents]

# Manually annotated sentences
manual_annotations = [
    ("2011-05-04T07:24:07Z -- http://www.bloomberg.com/news/2011-05-04/bbva-is-said-to-announce-falling-profit-on-thursday.html", []),
    ("Net income probably declined to 1.08 billion euros ($1.6 billion) from 1.24 billion euros a year earlier, according to the average estimate in a Bloomberg survey of six analysts.", [("Net income", "MONEY"), ("euros", "MONEY"), ("Bloomberg", "ORG"), ("six analysts", "ORG")]),
    ("The Bilbao, Spain-based lender is scheduled to report first-quarter results tomorrow before the stock market opens in Spain.", [("Bilbao", "GPE"), ("Spain", "GPE")]),
    ("Spanish banks have been hurt by weakening credit demand and souring loans in a domestic economy that’s still struggling to emerge from recession.", [("Spanish banks", "ORG")]),
    ("BBVA, led by Chairman Francisco Gonzalez, completed its $5.8 billion purchase of a 24.9 percent stake in Turkiye Garanti Bankasi AS (GARAN) in March as it expands outside Spain, adding Turkey to other emerging markets such as Mexico.", [("BBVA", "ORG"), ("Francisco Gonzalez", "PERSON"), ("$5.8 billion", "MONEY"), ("Turkiye Garanti Bankasi AS (GARAN)", "ORG"), ("March", "TIME"), ("Spain", "GPE"), ("Turkey", "GPE"), ("Mexico", "GPE")]),
    ("“The news from Spain won’t be good, but there probably won’t be nasty surprises while Mexico is showing interesting growth,” said Pablo Garcia, head of equities at Oddo Sociedad de Valores in Madrid.", [("Spain", "GPE"), ("Mexico", "GPE"), ("Pablo Garcia", "PERSON"), ("Oddo Sociedad de Valores", "ORG"), ("Madrid", "GPE")]),
    ("“All the big Spanish companies, and that includes BBVA, are at pains now to show that Spain is a diminishing part of their business.” BBVA shares have gained 14 percent this year, compared with a 2.9 percent advance in the 48-member Bloomberg Europe Banks and Financial Services Index.", [("Spanish companies", "ORG"), ("BBVA", "ORG"), ("Spain", "GPE"), ("BBVA", "ORG"), ("Bloomberg", "ORG"), ("Europe", "GPE")]),
    ("Banco Santander SA (SAN), Spain’s largest bank, which on April 28 reported a 5 percent drop in first-quarter profit, has risen 5.3 percent in 2011.", [("Banco Santander SA (SAN)", "ORG"), ("Spain", "GPE"), ("April 28", "TIME"), ("2011", "TIME")]),
    ("Mexican Gains Profit from BBVA’s Spain-dominated Iberian business may have dropped 33 percent from a year earlier to 394 million euros, according to estimates by Banco BPI SA (BPI) analyst Carlos Joaquim Peixoto.", [("BBVA", "ORG"), ("Spain", "GPE"), ("Iberian business", "ORG"), ("Banco BPI SA (BPI)", "ORG"), ("Carlos Joaquim Peixoto", "PERSON")]),
    ("Spain and Portugal together accounted for 45 percent of group profit in 2010.", [("Spain", "GPE"), ("Portugal", "GPE"), ("2010", "TIME")])
]

# Retrieve 10 random samples from the dataset
sample_data = dataset["train"].select(range(10))

# Store annotations and predictions
annotations = []

for i, example in enumerate(sample_data):
    text = example["sentence"]
    
    # Get the manual annotations for the text
    manual_annotation = manual_annotations[i][1]
    
    # Get the model's predictions
    model_prediction = get_entities_from_text(text)
    
    # Store both manual annotations and model predictions
    annotations.append({
        "text": text,
        "manual_label": manual_annotation,
        "model_prediction": model_prediction
    })

# Convert to DataFrame for easy viewing
df_manual = pd.DataFrame(annotations)
print(df_manual)